# Yolo Model evaluation

In this Notebook the Model prediction will be analyst

## Own Metric

RAAD measures the difference between the predicted affected area (sum of bounding box areas) and the actual affected area. It accounts for overlapping bounding boxes to avoid double-counting.

RAAD = (∣Predicted Area − True Area∣) / True Area

RAAD = 0: Perfect match between predicted and actual affected area.

RAAD > 0: Model either overestimates or underestimates the affected area.

RAAD > 1: Significant discrepancy between prediction and ground truth.

This metric helps evaluate how well the model estimates the severity of the disease in the image.

## Setup

In [2]:
!pip install -U ultralytics wandb
!pip install dotenv

  Using cached wandb-0.19.11-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached setproctitle-1.3.6-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.8 MB/s eta 0:00:00
Using cached wandb-0.19.11-py3-none-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.4 MB)
Using cached docker_pycreds-0.4.0-py2.py3-none-any.whl (9.0 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (63.0 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Using cached setproctitle-1.3.6-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.many

In [3]:
!yolo settings wandb=True
# check the .env file 
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get and print the WANDB_API_KEY
wandb_api_key = os.getenv("WANDB_API_KEY")
print(f"WANDB_API_KEY: [{wandb_api_key[:4]}...]")

WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at '/home/jovyan/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
JSONDict("/home/jovyan/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection/notebooks/datasets",
  "weights_dir": "weights",
  "runs_dir": "runs",
  "uuid": "8a115bbf5049f0fe55cf2ccd8be54ca8bfded6b963fd272724a959bb525556d2",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": false,
  "wandb": true,
  "vscode_msg": true,
  "openvino_msg": tr

In [4]:
import wandb
wandb.login(key=wandb_api_key)

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: Currently logged in as: rueedi-tobias (rueedi-tobias-hochschule-luzern) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import os
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from tqdm import tqdm
import wandb
import contextlib
import logging
from PIL import Image


from raad import raad_metric, calculate_area

SPLIT = "SID01"

PROJECT_NAME = "maize_disease_detection"
EVAL_PROJECT_NAME = "maize_evaluation"
PROJECT_ROOT = Path("/home/jovyan/DSPRO2/M-AI-ZE-Maize-diseases-detection")
DATA_PATH = Path(f"/exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/{SPLIT}")
TEST_IMAGES_PATH = DATA_PATH / "images/test"
TEST_LABELS_PATH = DATA_PATH / "labels/test"
METRIC_OUTPUT_PATH = TEST_LABELS_PATH / "raad_results.csv"
EVAL_OUTPUT_PATH = PROJECT_ROOT / "evaluation_results.csv"

print(f"Evaluating on split: {SPLIT}")
print(f"Data path: {DATA_PATH}")


Evaluating on split: SID01
Data path: /exchange/dspro2/M-AI-ZE/data/adjusted/1.1/splits/SID01


## RAAD Functions

In [ ]:
def yolo_to_xyxy(box, img_width, img_height):
    x_center, y_center, width, height = box
    x1 = int((x_center - width / 2) * img_width)
    y1 = int((y_center - height / 2) * img_height)
    x2 = int((x_center + width / 2) * img_width)
    y2 = int((y_center + height / 2) * img_height)
    return x1, y1, x2, y2

def calculate_area(box):
    x1, y1, x2, y2 = box
    width = max(0, x2 - x1)
    height = max(0, y2 - y1)
    return width * height

def intersection_area(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 < x2 and y1 < y2:
        return (x2 - x1) * (y2 - y1)
    return 0

def total_area(boxes):
    total = 0
    overlap_areas = set()

    for i, box1 in enumerate(boxes):
        area = calculate_area(box1)
        for j, box2 in enumerate(boxes):
            if i < j:
                overlap = intersection_area(box1, box2)
                if overlap > 0:
                    overlap_areas.add(overlap)

        total += area
    total -= sum(overlap_areas)
    return total


def raad_metric(pred_boxes, true_boxes, img_width, img_height):
    pred_xyxy = [yolo_to_xyxy(box, img_width, img_height) for box in pred_boxes]
    pred_area = total_area(pred_xyxy)
    true_area = total_area(true_boxes)
    epsilon = 1e-6
    return abs(pred_area - true_area) / max(true_area, epsilon)


## Load a Model